In [11]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras._tf_keras.keras.datasets import cifar10
from keras._tf_keras.keras.preprocessing.image import ImageDataGenerator
from keras._tf_keras.keras.models import Sequential
from keras._tf_keras.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras._tf_keras.keras.optimizers import Adam, RMSprop, SGD
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from flask import Flask, request, render_template
from sklearn.metrics import confusion_matrix

In [2]:
batch_size = 32 
num_classes = 10
#epochs = 1600
data_augmentation = True
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
# Normalize the input data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

In [4]:
# Define the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

C:\Users\KIIT0001\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
# Compile the model with different hyperparameters
learning_rates = [0.01, 0.1]
batch_sizes = [16, 32]

In [6]:
for lr in learning_rates:
    for bs in batch_sizes:
        model.compile(loss='categorical_crossentropy',
            optimizer="Adam",
            metrics=['accuracy'])
        m1=model.fit(x_train, y_train,
            batch_size=bs,
            epochs=50,
            validation_data=(x_test, y_test),
            shuffle=True)

Epoch 1/50
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 63s 16ms/step - accuracy: 0.3669 - loss: 1.7176 - val_accuracy: 0.5726 - val_loss: 1.2226
Epoch 2/50
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 51s 16ms/step - accuracy: 0.6062 - loss: 1.1123 - val_accuracy: 0.6547 - val_loss: 0.9939
Epoch 3/50
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 53s 17ms/step - accuracy: 0.6796 - loss: 0.9220 - val_accuracy: 0.6703 - val_loss: 0.9528
Epoch 4/50
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 67s 22ms/step - accuracy: 0.7207 - loss: 0.8049 - val_accuracy: 0.7004 - val_loss: 0.8655
Epoch 5/50
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 51s 16ms/step - accuracy: 0.7483 - loss: 0.7252 - val_accuracy: 0.7036 - val_loss: 0.8714
Epoch 6/50
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 52s 17ms/step - accuracy: 0.7676 - loss: 0.6630 - val_accuracy: 0.7150 - val_loss: 0.8498
Epoch 7/50
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 46s 15ms/step - accuracy: 0.7910 - loss: 0.5938 - val_accuracy: 0.7113 - val_loss: 0.8567
Epoch 8/50
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 47s 15ms/step - accuracy: 0.8047 -

In [7]:
 # Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Learning Rate: {lr}, Batch Size: {bs}, Optimizer: {Adam}, Accuracy: {accuracy:.3f}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6974 - loss: 6.8195
Learning Rate: 0.1, Batch Size: 32, Optimizer: <class 'keras.src.optimizers.adam.Adam'>, Accuracy: 0.690


In [15]:
# Fine-tune the model using data augmentation and regularization
datagen = ImageDataGenerator(rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
datagen.fit(x_train)

In [23]:
model.add(Dropout(0.2))
model.compile(optimizer= 'adam' , loss= keras.losses.binary_crossentropy, metrics=['accuracy'])
m1 = model.fit(datagen.flow(x_train, y_train, batch_size=32), epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
   1/1563 ━━━━━━━━━━━━━━━━━━━━ 1:35:37 4s/step - accuracy: 0.1562 - loss: 1.4733

C:\Users\KIIT0001\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 70s 42ms/step - accuracy: 0.1441 - loss: 1.6514 - val_accuracy: 0.1623 - val_loss: 11.6326
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 69s 44ms/step - accuracy: 0.1152 - loss: 1.7997 - val_accuracy: 0.1135 - val_loss: 21.8212
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 66s 42ms/step - accuracy: 0.1066 - loss: 1.8139 - val_accuracy: 0.1004 - val_loss: 43.3032
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 72s 46ms/step - accuracy: 0.0991 - loss: 1.8214 - val_accuracy: 0.1009 - val_loss: 42.5993
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 68s 43ms/step - accuracy: 0.1012 - loss: 1.8249 - val_accuracy: 0.1009 - val_loss: 42.5808
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 70s 44ms/step - accuracy: 0.0973 - loss: 1.8334 - val_accuracy: 0.1005 - val_loss: 43.4047
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 69s 44ms/step - accuracy: 0.1012 - loss: 1.8225 - val_accuracy: 0.1002 - val_loss: 79.1674
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 68s 44ms/step - accuracy: 0.1001 - los